## 3. Neuronale Netze trainieren, testen und prüfen

In Kapitel 2 wurde nun ein vollständig lauffähiges Neuronales Netz erstellt. Wir können bei diesem Netz die Knotenanzahl, die Lernrate anpassen. Um ein solches Netz mit Trainings- und Testdaten zu befüllen, müssen noch ein paar weitere Punkte beachtet werden. Diese folgen in diesem Kapitel. 


Das Kapitel ist dabei in folgende Schritte aufgeteilt:
1. Daten einlesen
2. Feature-Daten (Eingabewerte) auf ein Niveau zwischen 0 und 1 skalieren und in einen Fließkommazahl-Array überführen
3. Label-Daten (Ausgabewerte) auf ein Niveau zwischen 0 und 1 skalieren und in einen Integer-Array überführen
4. Die Skalierung mittels einer For-Schleife für alle Eingabe- und Ausgabewerte durchführen
5. Epochen einführen. D.h. das Netz darf mehrmals das Training mit denselben Daten durchlaufen um sich zu optimieren
6. prozentuale Performancemessung durchführen. Basierend auf dem Label wird geprüft wie gut die Trefferquote war
7. Netz vollständig trainieren, testen und prüfen
8. Optimierungsmöglichkeiten für das Netz aufzeigen (Lernrate anpassen, Epochen anpassen, Netzstruktur ändern, Mehr Daten)

## 3.1 Daten einlesen

Um Daten einlesen zu können, müssen wir zunächst diese Datei aufbereitet haben. Wichtig hierbei sind folgende Punkte:

1. Keine Überschriften
2. Label in der ersten Spalte
3. Label ist eine Ziffer
4. Keine Kommatas für 1,5 € sondern 1.5. Wir nutzen die amerikanische Schreibweise. Die Trenner zwischen den Spalten wird hingegen mit Kommatas gemacht.

In [1]:
data_file = open("data/iris_dataset/iris_one.csv", 'r')
data_list = data_file.readlines()
data_file.close()

Lassen wir uns zunächst die Länge der Tabelle anzeigen. Also die Anzahl Zeilen.

In [2]:
#Anzeige der Länge der Liste
len(data_list)

1

Und nun geben wir den Inhalt der ersten Zeile (also die 0 eintragen) direkt hier aus.

In [3]:
#Inhalt der Liste ausgeben
data_list[0]

'1,7.9,3.8,6.4,2'

Jetzt mal eine andere Liste einladen. Z.B. schwertlilie_test oder schwertlilie_train oder mnist_dataset/mnist_train_100

## 3.2 Featuredaten skalieren

Die Featuredaten aus Zeile 3 sind in einem Niveau zwischen 0 und 5.7. Das ist nicht so geschickt für unser Netz, da wir Zahlen zwischen 0 und 1 erwarten. Daher müssen wir diese Zahlen auf dieses Niveau bringen. Damit wir jedoch keine 0 und auch keine 1 herausbekommen wenden wir einen Trick an.

In [4]:
import numpy

data_file = open("data/iris_dataset/iris_one.csv", 'r')
data_list = data_file.readlines()
data_file.close()

print(data_list)

['1,7.9,3.8,6.4,2']


Wie wir sehen ist die erste Zahl eine 1. Das ist das Label für diesen Datensatz. Die zweite Zahl ist eine 5,7 und zugleich die größte Zahl. Wenn wir den Ursprungsdatensatz anschauen und in Excel mit der MAX()-Funktion prüfen, so werden wir erkennen, dass die größte Zahl eine 7,9 ist. Dies ist also unser Ausgangsskalierungsfaktor für die Features.

In [5]:
all_values = data_list[0].split(',')

scaled_input = (numpy.asfarray(all_values[1:]) / 7.9 * 0.99) + 0.01

print(all_values)
print(scaled_input)

['1', '7.9', '3.8', '6.4', '2']
[1.         0.48620253 0.81202532 0.26063291]


In Zeile 5 haben wir folgendes getan:

1. Der Datensatz 0 wurde ausgewählt und dieser String nach den Kommatas mit der Funktion split() aufgetrennt. Das Ergebnis wird in all_values abgelegt.
2. In der nächsten Zeile passieren einige Dinge gleichzeitig. all_values[1:] bedeutet, dass alle Elemente ab der Position 1 betroffen sind. Also alle Elemente außer dem Label auf Position 0. 
3. Die Funktion numpy.asfarray() konvertiert den bisherigen Textstring in echte Zahlen. 
4. Diese Zahlen teilen wir durch die maximal mögliche Zahl (7,9).
5. Um die Werte aus 1. auf 0,0 bis 0,99 zu bekommen müssen diese Werte mit 0,99 multipliziert werden.
6. Anschließend werden die Werte mit 0,01 addiert, sodass wir keine reinen 0-Werte bekommen.

Der direkte Vergleich der Zahlen (noch als Textwert) und die skalierten Werte werden per Print ausgegeben.

## 3.3 Labelwerte skalieren

Weitestgehend analog gehen wir nun mit den Labeldaten vor.  

In [6]:
#output nodes is 3 (example)
onodes = 3
targets = numpy.zeros(onodes) + 0.01
targets[int(all_values[0])] = 0.99

print(all_values[0])
print(targets)

1
[0.01 0.99 0.01]


Was passiert nun in Zeile 6:

1. Die Anzahl an Ausgabeknoten wird auf 3 gesetzt. Das liegt daran, dass wir im IRIS-Datensatz nur drei verschiedene Blütensorten haben.
2. numpy.zeros() erzeugt ein mit Nullen gefülltes Array. Größe und Gestalt des Arrays mit der Länge "onodes" wird als Parameter übergeben. Hinzuaddiert wird 0,01 um reine Nullwerte zu vermeiden.
3. Hier passiert nun wieder etwas mehr:
4. all_values[0] ist das Label/die Kennung des Datensatzes. In unserem Fall trägt das Label eine 1. Es ist also die zweite Blütensorte mit der Kennung und dem Array-Index 1 (möglich wären: 0, 1, 2).
5. Diese Kennung wird ein eine Ganzzahl gewandelt und 
6. mit dem array-index verbunden. 

## 3.4 Skalierung und Konvertierung auf alle Labels und Featuredaten anwenden

Im obigen Beispiel hatten wir nur jeweils einen Datensatz eingelesen. Wir sollten dies jedoch sowohl für den Trainingsdatensatz wie auch unseren Testdatensatz anwenden. Dies können wir mit einer For-Schleife elegant lösen.

In [7]:
import numpy

output_nodes = 3

training_data_file = open("data/iris_dataset/iris_train.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

In [8]:
for record in training_data_list:
    # split the record by the ',' commas
    all_values = record.split(',')
    # scale and shift the inputs
    inputs = (numpy.asfarray(all_values[1:]) / 7.9*0.99) + 0.01
    # create the target output values (all 0.01, except the desired label which is 0.99)
    targets = numpy.zeros(output_nodes) + 0.01
    # all_values[0] is the target label for this record
    targets[int(all_values[0])] = 0.99
    # n.train ist noch auskommentiert, da dies später auf unser Netz-Objekt n angewandt werden soll.
    #n.train(inputs, targets)
    pass

Neben den bekannten Elementen aus den vorherigen Schritten ist nun ein weiterer Punkt hinzugekommen. Die For-Schleife durchläuft die training_data_list solange bis kein Datensatz mehr auszulesen ist. Jeder Datensatz wird dabei einzeln mit obigen Schritten bearbeitet.

In [9]:
print(record)
print(inputs)

0,4.3,3,1.1,0.1

[0.54886076 0.38594937 0.1478481  0.02253165]


In [10]:
print(targets)

[0.99 0.01 0.01]


Das Gleiche machen wir dann noch mit den Testdaten.

In [11]:
import numpy

test_data_file = open("data/iris_dataset/iris_test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

In [12]:
for record in test_data_list:
    # split the record by the ',' commas
    all_values = record.split(',')
    # correct answer is first value
    correct_label = int(all_values[0])
    # scale and shift the inputs
    inputs = (numpy.asfarray(all_values[1:]) / 7.9 * 0.99) + 0.01    
    pass

In [13]:
print(correct_label)
print(inputs)

2
[0.68670886 0.38594937 0.57392405 0.19797468]


Ähnlichkeiten zum Trainingsdatensatz sind vorhanden. Neu ist, dass wir das Label (also den korrekten Wert) in die Variable correct_label schreiben.

Warum werden immer nur ein Wert ausgegeben? Das liegt daran, dass wir mit der For-Schleife den kompletten Datensatz mit 130 Trainingsdatensatz und 20 Testdatensätze zwar durchlaufen. Jedoch werden diese nicht wie zuvor in Listen abgelegt. Wir berechnen hier lediglich die entsprechenden skalierten Werte um diese an das Netz in der jeweiligen Funktion des Trainings oder Tests zu übergeben.

## 3.5 Epochen

Epochen sind Trainingswiederholungen. Mit weiteren Anzahl Epochen wird derselbe Trainingsdatensatz erneut durchlaufen. Dies dient dazu den Gradientenabstieg zu verfeinern. Siehe auch folgende Abbildung.

<center><img src="files/gradientenverfahren_alpha_sprung.png" "Test" width=600px></center>

Epochen benötigen wir demnach für den Trainingsdatensatz. Dies ist mittels einer weiteren For-Schleife schnell getan.

In [14]:
import numpy

# epochs is the number of times the training data set is used for training
epochs = 3

output_nodes = 3

training_data_file = open("data/iris_dataset/iris_train.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

# train the neural network
for e in range(epochs):
    # go through all records in the training data set
    for record in training_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 7.9*0.99) + 0.01
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        # n.train ist noch auskommentiert, da dies später auf unser Netz-Objekt n angewandt werden soll.
        #n.train(inputs, targets)
        pass
    pass

## 3.6 Performancemessung

Um die Qualität und die Aussagekraft unseres Netzes messen zu können, werden die durch das nun gewichtete Netz ermittelten Aussagewerte mit den tatsächlichen Labels verglichen. In unserem Beispiel-Datensatz haben wir wie oben erwänht 20 Testdatensätze. Sollten 10 korrekt sein und 10 falsch, so werden wir eine Performance von 0,5 oder 50% bekommen. Um dies zu ermitteln gehen wir in den Testdatensatz und schreiben jedes Mal mit was das Netz gemutmaßt hat und was tatsächlich korrekt war. Dies schreiben wir in eine temporäre Liste und am Ende wird verglichen.

In [15]:
import numpy

test_data_file = open("data/iris_dataset/iris_test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()


# scorecard for how well the network performs, initially empty
scorecard = []

# go through all the records in the test data set
for record in test_data_list:
    # split the record by the ',' commas
    all_values = record.split(',')
    # correct answer is first value
    correct_label = int(all_values[0])
    # scale and shift the inputs
    inputs = (numpy.asfarray(all_values[1:]) / 7.9*0.99) + 0.01
    # query the network
    outputs = n.query(inputs)
    # the index of the highest value corresponds to the label
    label = numpy.argmax(outputs)
    # append correct or incorrect to list
    if (label == correct_label):
        # network's answer matches correct answer, add 1 to scorecard
        scorecard.append(1)
    else:
        # network's answer doesn't match correct answer, add 0 to scorecard
        scorecard.append(0)
        pass
    
    pass

NameError: name 'n' is not defined

Dieser Quellcode funktioniert leider nur, mit dem Netz. Was wir im Anschluß uns dann gleich mal ansehen werden.

Doch zunächst gehen wir auf die neuen Elemente ein.
1. scorecard[] --> Scorecard ist eine leere Trefferliste, welche nach jedem Datensatz aktualisiert wird.
2. Das Netz wählt den größten Wert der Ausgabeknoten als Antwort. Der Index dieses Knotens wird mit der Kennung, also dem Wert im Label verglichen numpy.argmax()
3. Mittels if-else wird geprüft, ob das Label korrekt vorhergesagt wurde oder nicht. Entsprechend wird der Trefferliste eine 1 (Treffer korrekt) oder eine 0 (Treffer falsch) eingetragen.

Das Beste wäre wir überführen unsere Erkenntnisse nun vollständig in das Netz.

## 3.7 vollständiges Netz

In [16]:
#######################################################################################
##################### 1. Import der benötigten Bibs ###################################
import numpy
# scipy.special for the sigmoid function expit()
import scipy.special
# library for plotting arrays
#import matplotlib.pyplot
# ensure the plots are inside this notebook, not an external window
#%matplotlib inline

#######################################################################################
##################### 2. Anpassung der Variablen ######################################
# number of input, hidden and output nodes
input_nodes = 4
hidden_nodes = 10
output_nodes = 3

# learning rate
learning_rate = 0.3

# epochs is the number of times the training data set is used for training
epochs = 3

# load the training data CSV file into a list
training_data_file = open("data/iris_dataset/iris_train.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

# load the test data CSV file into a list
test_data_file = open("data/iris_dataset/iris_test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

#######################################################################################
##################### 3. Klasse des Neuronalen Netzes #################################
# neural network class definition
class neuralNetwork:
    
    
    # initialise the neural network
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        # set number of nodes in each input, hidden, output layer
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        # link weight matrices, wih and who
        # weights inside the arrays are w_i_j, where link is from node i to node j in the next layer
        # w11 w21
        # w12 w22 etc 
        self.wih = numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.onodes, self.hnodes))

        # learning rate
        self.lr = learningrate
        
        # activation function is the sigmoid function
        self.activation_function = lambda x: scipy.special.expit(x)
        
        pass

    
    # train the neural network
    def train(self, inputs_list, targets_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        # output layer error is the (target - actual)
        output_errors = targets - final_outputs
        # hidden layer error is the output_errors, split by weights, recombined at hidden nodes
        hidden_errors = numpy.dot(self.who.T, output_errors) 
        
        # update the weights for the links between the hidden and output layers
        self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        
        # update the weights for the links between the input and hidden layers
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        
        pass

    
    # query the neural network
    def query(self, inputs_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

#######################################################################################
##################### 4. Erstellen eines Objekts der obigen Klasse ####################
    
# create instance of neural network
n = neuralNetwork(input_nodes,hidden_nodes,output_nodes, learning_rate)

#######################################################################################
##################### 5. Das Netz basierend auf den Epochen trainieren ################
 

# train the neural network
for e in range(epochs):
    # go through all records in the training data set
    for record in training_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 7.9*0.99) + 0.01
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)
        pass
    pass

#######################################################################################
##################### 6. Das Netz auf Basis der Testdaten prüfen ######################

# test the neural network

# scorecard for how well the network performs, initially empty
scorecard = []

# go through all the records in the test data set
for record in test_data_list:
    # split the record by the ',' commas
    all_values = record.split(',')
    # correct answer is first value
    correct_label = int(all_values[0])
    # scale and shift the inputs
    inputs = (numpy.asfarray(all_values[1:]) / 7.9*0.99) + 0.01
    # query the network
    outputs = n.query(inputs)
    # the index of the highest value corresponds to the label
    label = numpy.argmax(outputs)
    # append correct or incorrect to list
    if (label == correct_label):
        # network's answer matches correct answer, add 1 to scorecard
        scorecard.append(1)
    else:
        # network's answer doesn't match correct answer, add 0 to scorecard
        scorecard.append(0)
        pass
    print(label,correct_label,scorecard)
    pass

#######################################################################################
##################### 7. Ausgabe der Genauigkeit des Netzes (Performance) #############

# calculate the performance score, the fraction of correct answers
scorecard_array = numpy.asarray(scorecard)
print ("performance = ", scorecard_array.sum() / scorecard_array.size)

#######################################################################################
#######################################################################################

0 2 [0]
0 2 [0, 0]
0 2 [0, 0, 0]
0 1 [0, 0, 0, 0]
0 0 [0, 0, 0, 0, 1]
0 0 [0, 0, 0, 0, 1, 1]
0 2 [0, 0, 0, 0, 1, 1, 0]
0 2 [0, 0, 0, 0, 1, 1, 0, 0]
0 2 [0, 0, 0, 0, 1, 1, 0, 0, 0]
0 2 [0, 0, 0, 0, 1, 1, 0, 0, 0, 0]
0 2 [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]
0 1 [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0]
0 2 [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]
0 2 [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
0 2 [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0 2 [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0 2 [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0 0 [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
0 0 [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
0 2 [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0]
performance =  0.2


Mit einer Performance von 20% ist das Netz noch optimierungsfähig. Trotzdem wollen wir mal sehen wie gut ein spezieller Datensatz vorhergesagt wird.

In [17]:
# Ersten Datensatz auslesen
all_values=test_data_list[0].split(',')
# Ausgabe des Labels
print(all_values[0])

2


In [18]:
n.query((numpy.asfarray(all_values[1:]) / 7.9 * 0.99) + 0.01)

array([[0.76319629],
       [0.14158856],
       [0.16869803]])

Das Netz liest zunächst das Label aus. Eine 2. In der anschliessenden Vorhersage liegt die 2 bei 17%. Das Netz würde hier eine 0 mit 75% vorhersagen.

## 3.8 Optimierungsmöglichkeiten

**Lernrate**: Unter Umständen muß die Lernrate zu einem späteren Zeitpunkt angepasst werden. Diese Lernrate wird notwendig um den Sprung im Gradientenverfahren nicht zu klein oder zu groß werden zu lassen.

<center><img src="files/gradientenverfahren_alpha_sprung.png" "Test" width=600px></center>

**Trainingswiederholungen**: Mit weiteren Anzahl Epochen wird derselbe Trainingsdatensatz erneut durchlaufen. Dies dient dazu den Gradientenabstieg zu verbessern.

**Mehr Daten**: Je größer die Datenmengen für das Training, desto genauer werden die Ergebnisse. 

**Netzstruktur ändern**: Anzahl der Knoten der versteckten Schicht anpassen.

## 3.9 Netz optimieren

In [25]:
#######################################################################################
##################### 1. Import der benötigten Bibs ###################################
import numpy
# scipy.special for the sigmoid function expit()
import scipy.special
# library for plotting arrays
#import matplotlib.pyplot
# ensure the plots are inside this notebook, not an external window
#%matplotlib inline

#######################################################################################
##################### 2. Anpassung der Variablen ######################################
# number of input, hidden and output nodes
input_nodes = 4
hidden_nodes = 10
output_nodes = 3

# learning rate
learning_rate = 0.3

# epochs is the number of times the training data set is used for training
epochs = 100

# load the training data CSV file into a list
training_data_file = open("data/iris_dataset/iris_train.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

# load the test data CSV file into a list
test_data_file = open("data/iris_dataset/iris_test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

#######################################################################################
##################### 3. Klasse des Neuronalen Netzes #################################
# neural network class definition
class neuralNetwork:
    
    
    # initialise the neural network
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        # set number of nodes in each input, hidden, output layer
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        # link weight matrices, wih and who
        # weights inside the arrays are w_i_j, where link is from node i to node j in the next layer
        # w11 w21
        # w12 w22 etc 
        self.wih = numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.onodes, self.hnodes))

        # learning rate
        self.lr = learningrate
        
        # activation function is the sigmoid function
        self.activation_function = lambda x: scipy.special.expit(x)
        
        pass

    
    # train the neural network
    def train(self, inputs_list, targets_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        # output layer error is the (target - actual)
        output_errors = targets - final_outputs
        # hidden layer error is the output_errors, split by weights, recombined at hidden nodes
        hidden_errors = numpy.dot(self.who.T, output_errors) 
        
        # update the weights for the links between the hidden and output layers
        self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        
        # update the weights for the links between the input and hidden layers
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        
        pass

    
    # query the neural network
    def query(self, inputs_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

#######################################################################################
##################### 4. Erstellen eines Objekts der obigen Klasse ####################
    
# create instance of neural network
n = neuralNetwork(input_nodes,hidden_nodes,output_nodes, learning_rate)

#######################################################################################
##################### 5. Das Netz basierend auf den Epochen trainieren ################
 

# train the neural network
for e in range(epochs):
    # go through all records in the training data set
    for record in training_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 7.9*0.99) + 0.01
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)
        pass
    pass

#######################################################################################
##################### 6. Das Netz auf Basis der Testdaten prüfen ######################

# test the neural network

# scorecard for how well the network performs, initially empty
scorecard = []

# go through all the records in the test data set
for record in test_data_list:
    # split the record by the ',' commas
    all_values = record.split(',')
    # correct answer is first value
    correct_label = int(all_values[0])
    # scale and shift the inputs
    inputs = (numpy.asfarray(all_values[1:]) / 7.9*0.99) + 0.01
    # query the network
    outputs = n.query(inputs)
    # the index of the highest value corresponds to the label
    label = numpy.argmax(outputs)
    # append correct or incorrect to list
    if (label == correct_label):
        # network's answer matches correct answer, add 1 to scorecard
        scorecard.append(1)
    else:
        # network's answer doesn't match correct answer, add 0 to scorecard
        scorecard.append(0)
        pass
    print(label,correct_label,scorecard)
    pass

#######################################################################################
##################### 7. Ausgabe der Genauigkeit des Netzes (Performance) #############

# calculate the performance score, the fraction of correct answers
scorecard_array = numpy.asarray(scorecard)
print ("performance = ", scorecard_array.sum() / scorecard_array.size)

#######################################################################################
#######################################################################################

2 2 [1]
2 2 [1, 1]
2 2 [1, 1, 1]
1 1 [1, 1, 1, 1]
0 0 [1, 1, 1, 1, 1]
0 0 [1, 1, 1, 1, 1, 1]
2 2 [1, 1, 1, 1, 1, 1, 1]
2 2 [1, 1, 1, 1, 1, 1, 1, 1]
2 2 [1, 1, 1, 1, 1, 1, 1, 1, 1]
2 2 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
2 2 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
1 1 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
2 2 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
2 2 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
2 2 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
2 2 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
2 2 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
0 0 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
0 0 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
2 2 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
performance =  1.0


In [26]:
# Ersten Datensatz auslesen
all_values=test_data_list[0].split(',')
# Ausgabe des Labels
print(all_values[0])
n.query((numpy.asfarray(all_values[1:]) / 7.9 * 0.99) + 0.01)

2


array([[0.03119014],
       [0.01436555],
       [0.93431681]])

## 4.0 Maximalwert der Sets auslesen und automatisch eintragen

Der Maximalwert mußte bislang manuell in den Daten-Dateien ausgelesen werden. Dies kann mittels der numpy.amax()-Funktion automatisiert werden, sodass dieser Wert nicht hardcoded in den Quelltext eingetragen werden muß. Siehe folgenden Code bei 2.1.

In [30]:
#######################################################################################
##################### 1. Import der benötigten Bibs ###################################
import numpy
# scipy.special for the sigmoid function expit()
import scipy.special
# library for plotting arrays
#import matplotlib.pyplot
# ensure the plots are inside this notebook, not an external window
#%matplotlib inline

#######################################################################################
##################### 2. Anpassung der Variablen ######################################
# number of input, hidden and output nodes
input_nodes = 4
hidden_nodes = 10
output_nodes = 3

# learning rate
learning_rate = 0.3

# epochs is the number of times the training data set is used for training
epochs = 100

# load the training data CSV file into a list
training_data_file = open("data/iris_dataset/iris_train.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

# load the test data CSV file into a list
test_data_file = open("data/iris_dataset/iris_test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

#######################################################################################
##################### 2.1 Maximalfeaturewert auslesen #################################
# max_test_train_set um alle Werte in eine Liste zu schreiben
max_test_train_set = []

# go through all the records in the test data set
for record in test_data_list:
    # split the record by the ',' commas
    all_values = record.split(',')
    # correct answer is first value
    correct_label = int(all_values[0])
    # scale and shift the inputs
    inputs = numpy.asfarray(all_values[1:])
    max_test_train_set.append(inputs)  
    pass

for record in training_data_list:
    # split the record by the ',' commas
    all_values = record.split(',')
    # correct answer is first value
    correct_label = int(all_values[0])
    # scale and shift the inputs
    inputs = numpy.asfarray(all_values[1:])
    max_test_train_set.append(inputs)  
    pass

#print(max_test_train_set)

max_test_train_value=numpy.amax(max_test_train_set)

#print(max_test_value)

#######################################################################################
##################### 3. Klasse des Neuronalen Netzes #################################
# neural network class definition
class neuralNetwork:
    
    
    # initialise the neural network
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        # set number of nodes in each input, hidden, output layer
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        # link weight matrices, wih and who
        # weights inside the arrays are w_i_j, where link is from node i to node j in the next layer
        # w11 w21
        # w12 w22 etc 
        self.wih = numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.onodes, self.hnodes))

        # learning rate
        self.lr = learningrate
        
        # activation function is the sigmoid function
        self.activation_function = lambda x: scipy.special.expit(x)
        
        pass

    
    # train the neural network
    def train(self, inputs_list, targets_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        # output layer error is the (target - actual)
        output_errors = targets - final_outputs
        # hidden layer error is the output_errors, split by weights, recombined at hidden nodes
        hidden_errors = numpy.dot(self.who.T, output_errors) 
        
        # update the weights for the links between the hidden and output layers
        self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        
        # update the weights for the links between the input and hidden layers
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        
        pass

    
    # query the neural network
    def query(self, inputs_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

#######################################################################################
##################### 4. Erstellen eines Objekts der obigen Klasse ####################
    
# create instance of neural network
n = neuralNetwork(input_nodes,hidden_nodes,output_nodes, learning_rate)

#######################################################################################
##################### 5. Das Netz basierend auf den Epochen trainieren ################
 

# train the neural network
for e in range(epochs):
    # go through all records in the training data set
    for record in training_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / max_test_train_value*0.99) + 0.01
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)
        pass
    pass

#######################################################################################
##################### 6. Das Netz auf Basis der Testdaten prüfen ######################

# test the neural network

# scorecard for how well the network performs, initially empty
scorecard = []

# go through all the records in the test data set
for record in test_data_list:
    # split the record by the ',' commas
    all_values = record.split(',')
    # correct answer is first value
    correct_label = int(all_values[0])
    # scale and shift the inputs
    inputs = (numpy.asfarray(all_values[1:]) / max_test_train_value*0.99) + 0.01
    # query the network
    outputs = n.query(inputs)
    # the index of the highest value corresponds to the label
    label = numpy.argmax(outputs)
    # append correct or incorrect to list
    if (label == correct_label):
        # network's answer matches correct answer, add 1 to scorecard
        scorecard.append(1)
    else:
        # network's answer doesn't match correct answer, add 0 to scorecard
        scorecard.append(0)
        pass
    print(label,correct_label,scorecard)
    pass

#######################################################################################
##################### 7. Ausgabe der Genauigkeit des Netzes (Performance) #############

# calculate the performance score, the fraction of correct answers
scorecard_array = numpy.asarray(scorecard)
print ("performance = ", scorecard_array.sum() / scorecard_array.size)

#######################################################################################
#######################################################################################

2 2 [1]
2 2 [1, 1]
2 2 [1, 1, 1]
1 1 [1, 1, 1, 1]
0 0 [1, 1, 1, 1, 1]
0 0 [1, 1, 1, 1, 1, 1]
2 2 [1, 1, 1, 1, 1, 1, 1]
2 2 [1, 1, 1, 1, 1, 1, 1, 1]
2 2 [1, 1, 1, 1, 1, 1, 1, 1, 1]
2 2 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
2 2 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
1 1 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
2 2 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
2 2 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
2 2 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
2 2 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
2 2 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
0 0 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
0 0 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
2 2 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
performance =  1.0


In [32]:
##################### Testabfrage für neue eigene Eingangswerte ###########################################
##################### Mutmaßt das Netz die richtige Antwort? ##############################################
# wähle dazu einen passenden Datensatz und trage die entsprechenden Featurewerte hier ein (bsp. Label=2 und Feature=5,2,3.5,1)

n.query([5,2,3.5,1])

array([[0.00640738],
       [0.00735859],
       [0.99657759]])